# Realized Covariance

Reads in a csv file containing intraday price data for 9 stocks and outputs a dataframe consisting of the vectorization of the realized covariance for each date provided

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('D:/Thesis/IntradayReturns.csv')
df.Datetime = pd.to_datetime(df.Datetime)
df = df[df.Datetime <= pd.to_datetime('2021-03-01', format = '%Y-%m-%d')]

In [3]:
dates = list(df.Datetime.dt.date.astype(str).unique())

In [4]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [5]:
df = df.set_index([df.Datetime.dt.date.astype(str), df.Datetime.dt.time.astype(str)])
df.drop('Datetime', axis = 1, inplace = True)
df = df.set_index(df.index.rename(['Date', 'Time']))

In [6]:
# Create dictionary of dataframes where date is key and values are prices for each stock at each time
dfs = {} 
for date in dates:
    dfs[date] = df.loc[date]

In [7]:
# Transform prices to log returns * 100
dfs = {date: 100* np.log(dfs[date]).diff().dropna() for date in dates}

In [8]:
# Calculate realized covariation for each date
cov_ests = {}
for date in dates:
    cov = np.zeros((9,9))
    X = dfs[date].values
    for t in range(77):
        cov += np.outer(X[t,:], X[t,:])
    cov_ests[date] = cov.ravel()

In [31]:
pd.DataFrame(cov_ests).T.to_csv('logIntradayRV.csv')